### Amazon S3

Amazon Simple Storage Service (Amazon S3) is an object storage service that offers industry-leading scalability, data availability, security, and performance. Customers of all sizes and industries can use Amazon S3 to store and protect any amount of data for a range of use cases, such as data lakes, websites, mobile applications, backup and restore, archive, enterprise applications, IoT devices, and big data analytics. Amazon S3 provides management features so that you can optimize, organize, and configure access to your data to meet your specific business, organizational, and compliance requirements.

PRACTICING:

In [ ]:
import boto3
import pandas as pd
import json

In [1]:
with open ('/run/media/roberto/black-box/.syek/aws/s33ding.json', "r") as f:
    cred = json.load(f)

s3 = boto3.client('s3', region_name='us-east-1',aws_access_key_id=cred.get("id"), aws_secret_access_key=cred.get("secret"))

In [2]:
s3.create_bucket(Bucket= "trash-456")

{'ResponseMetadata': {'RequestId': 'Q8X5RDKD2ERZGTEH',
  'HostId': 'NoTUDzWRXxy64Xlgy4kgDi/I36moY5y4Ns3WFdV8MqF+mjqcXH/BIurMs8ZsYl3d29BYgEyYQxU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NoTUDzWRXxy64Xlgy4kgDi/I36moY5y4Ns3WFdV8MqF+mjqcXH/BIurMs8ZsYl3d29BYgEyYQxU=',
   'x-amz-request-id': 'Q8X5RDKD2ERZGTEH',
   'date': 'Fri, 11 Nov 2022 02:42:34 GMT',
   'location': '/trash-456',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/trash-456'}

In [3]:
s3.delete_bucket(Bucket="trash-456")

{'ResponseMetadata': {'RequestId': 'Q8X1F2E4RYCRBXQ8',
  'HostId': 'iB0bA/gpuPA401s046PH8iI/vNvjWfc/ka3Wf1embd0pMe8OApXzhH+yVxY9OBNEDr5yiYCcwSk=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'iB0bA/gpuPA401s046PH8iI/vNvjWfc/ka3Wf1embd0pMe8OApXzhH+yVxY9OBNEDr5yiYCcwSk=',
   'x-amz-request-id': 'Q8X1F2E4RYCRBXQ8',
   'date': 'Fri, 11 Nov 2022 02:42:34 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [4]:

def printing_buckets():
    lst = []
    for bucket in  s3.list_buckets()['Buckets']:
        lst.append(bucket['Name'])
    print(f"My buckets: {lst}")

def search_bucket(bucket_nm):
    for bucket in s3.list_buckets()['Buckets']:
        if bucket_nm in bucket['Name']: 
          return bucket



In [5]:
fileSrc = './filesToUpload/mascot.jpeg'
keyName = 'linux.jpeg'
bucketName = 'roberto-server'

#uploading the files
s3.upload_file(Bucket = bucketName, Filename= fileSrc, Key = keyName)

#download the files
s3.download_file(Filename= keyName, Bucket= bucketName, Key = keyName)

#deleting object_key
s3.delete_object( Bucket=bucketName, Key=keyName)


FileNotFoundError: [Errno 2] No such file or directory: './filesToUpload/mascot.jpeg'

In [ ]:
# uploading the file with extra arg so that it can be public, for default aws files are upload as private for security reasons
s3.upload_file(
    Bucket = bucketName,
    Filename= fileSrc,
    Key = keyName, 
    ExtraArgs = {'ACL':'public-read'}
    )

# you can also make the entire bucket public with command bellow
s3.put_object_acl(Bucket= bucketName, Key = keyName, ACL = 'public-read')

url = 'https://{}.s3.amazonaws.com/{}'.format(bucketName, keyName); print(url)


In [ ]:
htmlFileName = 'home.html'
fileSrcHtml = './filesToUpload/'+ htmlFileName

# Upload the lines.html file to S3
s3.upload_file(Filename=fileSrcHtml, 
               # Set the bucket name
               Bucket=bucketName, Key=htmlFileName,
               # Configure uploaded file
               ExtraArgs = {
                 # Set proper content type
                 'ContentType':'text/html',
                 # Set proper ACL
                 'ACL': 'public-read'})

# Print the S3 Public Object URL for the new file.
print("http://{}.s3.amazonaws.com/{}".format(bucketName, htmlFileName))


In [ ]:

csvFile = './filesToUpload/campeonato-brasileiro-full.csv'
df = pd.read_csv(csvFile)

# upload a csv file as html
htmlFileName = 'campeonato-brasileiro-full.html'
fileSrcHtml = './filesToUpload/'+htmlFileName
df.to_html(fileSrcHtml, border=0, render_links=True)


# Upload the generated HTML to the gid-reports bucket
s3.upload_file(Filename= fileSrcHtml, Key=htmlFileName, Bucket=bucketName,
 ExtraArgs = {'ContentType': 'text/html', 'ACL': 'public-read'})

print("http://{}.s3.amazonaws.com/{}".format(bucketName, htmlFileName))


In [ ]:

# copy_obj
def copy_obj(bucket_orig = "", obj_orig = "", bucket_dest = "", obj_dest = ""):
    s3 = boto3.resource('s3')
    copy_source = {'Bucket': bucket_orig, 'Key': obj_orig}
    bucket = s3.Bucket(bucket_dest)
    bucket.copy(copy_source, obj_dest)

def check_if_obj_exist(bucket_nm, key_nm):
    s3 = boto3.resource('s3', region_name='us-east-1')
    try:
        s3.Object(bucket_nm, key_nm).load()
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
        # The object does not exist.
            return False
        else:
        # Something else has gone wrong.
            raise
    else:
    # The object does exist.
        return key